In [1]:
import pandas as pd
import openai
import os
from annotation_utils import get_val_scores, make_transcript
from AptTagger import AptTagger
from TruthLabeler import TruthLabeler

openai.api_key_path = "/home/alex/.personal/openAIkey"  # read protected to my 174 account

/mnt/ssd-2/spar/alexm/miniconda3/envs/dlkb/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
val_ids, val_scores = get_val_scores()

In [3]:
val_df = pd.read_json("data/label-validation/transcripts_combined.json")
val_df.index = val_df["message_id"]
scores = [val_scores[val_ids.index(id)] for id in val_df.index]
val_df["score"] = scores

In [4]:
# texts = []
# for path in os.listdir("data/oasst/transcripts/original"):
#     with open(os.path.join("data/oasst/transcripts/original", path)) as f:
#         texts.append(f.read())

# text = "\n\n\n".join(texts)
# raw_df = pd.DataFrame()
# raw_df["message_id"] = get_message_ids(text)
# raw_df["user_text"] = get_prompter_texts(text)
# raw_df["assitant_text"] = get_assistant_texts(text)
# raw_df.index = raw_df["message_id"]
# print(len(raw_df))

In [5]:
apt_tagger = AptTagger(
    model_name="data/apt-tagger-deberta-xlarge-mnli-s63day9p",
    device="cuda"
)

In [6]:
raw_df = val_df

In [7]:
raw_df["tag_mask"] = apt_tagger.tag(raw_df["assistant_text"])
raw_df["annotated_assistant_text"] = raw_df.apply(lambda x: apt_tagger.annotate(x["assistant_text"], x["tag_mask"]), axis=1)
raw_df["annotated_transcript"] = raw_df.apply(make_transcript, axis=1)
raw_df.head()

,prompt,response,parent_id,parent_text,role,synthetic,model,assistant_text,user_text,message_id,prev_messages,rank,score,tag_mask,annotated_assistant_text,annotated_transcript
message_id,,,,,,,,,,,,,,,,
87cdf227-c048-4ccb-a790-0f44dc5ebb6b,A chat between a curious human and an artifici...,"As an artificial intelligence language model, ...",6faa03e5-5f0c-483c-9d22-a15505a2b821,Where was your last vaccation?,assistant,True,keyfan/bloomz-rlhf(max_length=512;temperature=...,"As an artificial intelligence language model, ...",Where was your last vaccation?,87cdf227-c048-4ccb-a790-0f44dc5ebb6b,[Where was your last vaccation?],NaN,"[Certainly True, Certainly True, Certainly Tru...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","As an artificial intelligence language model, ...",USER: Where was your last vaccation?\n\nASSIST...
fd9fa103-6988-4d4c-b6de-2e7a0ac8b5a9,A chat between a curious human and an artifici...,Joseph Scaliger (1557-1636) was an Italian sch...,2f2e3cfd-857f-41b5-9824-0b62f7dd374c,Describe Joseph Scaliger and his contributions...,assistant,True,keyfan/bloomz-rlhf(max_length=512;temperature=...,Joseph Scaliger (1557-1636) was an Italian sch...,Describe Joseph Scaliger and his contributions...,fd9fa103-6988-4d4c-b6de-2e7a0ac8b5a9,[Describe Joseph Scaliger and his contribution...,NaN,"[Certainly False, True, True, Certainly True, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Joseph Scaliger (1557-1636) was an Italian sch...,USER: Describe Joseph Scaliger and his contrib...
69f27058-6253-42bc-892c-5f8a95870ec1,A chat between a curious human and an artifici...,Certainly! A well-chosen color palette is one ...,89b13be2-c82f-493a-9258-1d690440e451,"I am designing a website, can you give me a co...",assistant,True,keyfan/bloomz-rlhf(max_length=512;temperature=...,Certainly! A well-chosen color palette is one ...,"I am designing a website, can you give me a co...",69f27058-6253-42bc-892c-5f8a95870ec1,"[I am designing a website, can you give me a c...",NaN,"[N/A, N/A, N/A, N/A, N/A, N/A, N/A, N/A, N/A, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Certainly! A well-chosen color palette is one ...,"USER: I am designing a website, can you give m..."
b864688c-62cb-420d-bae1-e81677b4fded,A chat between a curious human and an artifici...,Lena was a happy-go-lucky suburban housewife w...,0cd84750-ccf6-4a8e-89bf-02e17a6d0eca,Write me a story synopsis for a novel where th...,assistant,True,keyfan/bloomz-rlhf(max_length=512;temperature=...,Lena was a happy-go-lucky suburban housewife w...,Write me a story synopsis for a novel where th...,b864688c-62cb-420d-bae1-e81677b4fded,[Write me a story synopsis for a novel where t...,NaN,"[N/A, N/A, N/A, N/A, N/A, N/A, N/A, N/A, N/A, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Lena was a happy-go-lucky suburban housewife w...,USER: Write me a story synopsis for a novel wh...
0412dc3f-ae56-4d83-b361-cb1fa7c248a6,A chat between a curious human and an artifici...,I apologize if my previous response did not ac...,dfe226cd-1663-45e3-ab80-7b6065da46ac,"I shouldn't need to provide context, that is w...",assistant,True,keyfan/bloomz-rlhf(max_length=512;temperature=...,I apologize if my previous response did not ac...,"I shouldn't need to provide context, that is w...",0412dc3f-ae56-4d83-b361-cb1fa7c248a6,[A ball was thrown on a planet with a density ...,NaN,"[N/A, True, N/A, False, Certainly False, N/A, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",I apologize if my previous response did not ac...,USER: A ball was thrown on a planet with a den...


In [8]:
from Retriever import Retriever
retriever = Retriever(
    use_IVF=True,
    encoder_name="BAAI/bge-small-en",
    pretrained_index_path='retrieval/20230601.en.wiki_embeddings_bge-small-en_IVF.index',
)

truth_labeler = TruthLabeler(
                retriever,
                model_name="gpt-3.5-turbo",
                temperature=1,
                n_samples=5,
                uncertainty_bias=0,
                na_bias=0,
)

Loading model... done.
Loading pretrained index... done.


AttributeError: 'NoneType' object has no attribute 'keys'

In [22]:
text = \
"""Joseph Scaliger (1557-1636) was an Italian scholar and theologian who made significant contributions to our current understanding of history[[1]].

Scaliger is known for emphasizing the importance of empirical evidence in historical research[[2]]. He rejected the traditional belief that religious beliefs were settled truths, arguing instead that they could evolve over time based on new information[[3]]. This view challenged earlier Catholic ideas about history as ordered by divine revelation[[4]].

Scaliger also developed the concept of "historical reason," or the use of rational argument to understand events from a purely secular perspective[[5]]. His works helped pave the way for the development of Enlightenment thought, which emphasized critical thinking and an empiricist approach to scientific inquiry[[6]].

Finally, Scaliger played a key role in promoting women's education throughout Europe during the Elizabethan era[[7]]. As president of the University of Paris he introduced several courses specifically designed to promote women’s learning including a classics course where female scholars were encouraged to study alongside their male counterparts[[8]].
###

First, decide whether each statement can even be given an accuracy label, writing "Determinable" if it can, and "N/A" if the statement doesn't make any claims that can be judged on their accuracy. Then, if the statement is "Determinable",  rate the extent to which it is true, accurate, and not misleading. Your rating must be one of "N/A", "Determinable - Certainly True", "Determinable - True", "Determinable - Uncertain", "Determinable - False", "Determinable - Certainly False". Ratings should be based on the quality of the new information introduced in a statement, and if anything in the statement is incorrect or misleading, the statement is considered incorrect.

Fill in the following list with your rating for each of the statements marked, without any explanations or anything other than the requested completions. If a Wikipedia excerpt would be helpful for you to rate the accuracy of the transcript, indicate so with a "Yes" in the field provided, otherwise "No". Only use Wikipedia in cases where you are unfamiliar with the subject."""
result = retriever(text, threshold=0.1)
print(result[0][0])
print(result[1])

'doubts about this matter do not deserve one word of rebuttal.'...[E]vidence of 'historical type' gives way to 'psychological' arguments... 'of sympathy': Is it necessary or not, to assume at the origin of all and every religion a historical figure?...[T]he question.. becomes 'placed in parenthesis,' which leads exegetes... to push the Jesus of history back into the shadows while vigorously spotlighting the Jesus of faith."

Bibliography

Key works
 L'énigme de Jésus (Mercure de France, 1923); transl. Winifred Stephens Whale, The Enigma of Jesus, (Watts, 1924)
 Le mystère de Jésus ("The Mystery of Jesus")  (F. Rieder, 1924). No English translation
 La Sagesse Juive:
[0.1761319]


In [24]:
out_df = truth_labeler.label(raw_df["message_id"], raw_df["annotated_transcript"], n_threads=10)
# NOTE: THERE ARE DUPLICATE IDS IN THE OUTPUT DF WHEN USING RETRIEVER SO THAT WE CAN COMPARE THE RESULTS
# TODO: Change this once I've finished validating the retriever
out_df_retr = out_df[out_df["used_retrieval"]]
retr_ids = set(out_df_retr["message_id"])
out_df_counter_retr = out_df[~((out_df["message_id"].isin(retr_ids)) & out_df["used_retrieval"])]
# remove all the rows with message_id in retr_ids but with used_retrieval==False
out_df = out_df[~(out_df["message_id"].isin(retr_ids) & (~out_df["used_retrieval"]))]

SKIPPING: scores must be one of ['N/A', 'Truth-apt - Certainly True', 'Truth-apt - True', 'Truth-apt - Uncertain', 'Truth-apt - False', 'Truth-apt - Certainly False'], but found ['[[1]] Truth-apt - Certainly True', '[[2]] Truth-apt - True']
SKIPPING: scores must be one of ['N/A', 'Truth-apt - Certainly True', 'Truth-apt - True', 'Truth-apt - Uncertain', 'Truth-apt - False', 'Truth-apt - Certainly False'], but found ['[[1]] Truth-apt - Uncertain', '[[2]] Truth-apt - True']
SKIPPING: scores must be one of ['N/A', 'Truth-apt - Certainly True', 'Truth-apt - True', 'Truth-apt - Uncertain', 'Truth-apt - False', 'Truth-apt - Certainly False'], but found ['[[1]] Truth-apt - True', '[[2]] Truth-apt - True', '[[3]] Truth-apt - True']
SKIPPING: scores must be one of ['N/A', 'Truth-apt - Certainly True', 'Truth-apt - True', 'Truth-apt - Uncertain', 'Truth-apt - False', 'Truth-apt - Certainly False'], but found ['[[1]] Truth-apt - False', '[[2]] Truth-apt - True', '[[3]] Truth-apt - True', '[[4]] T

In [27]:
labeled_df = out_df.copy()
labeled_df.index = out_df["message_id"]
labeled_df = labeled_df.drop(columns=["message_id", "annotated_transcript"])
df = labeled_df.join(raw_df, how="left")
df.head()

,idx,input,responses,prompt_tokens,completion_tokens,dollars,pred_scores,p_apts,p_trues,ann_count,message_id,user_text,assitant_text,tag_mask,annotated_assistant_text,annotated_transcript
message_id,,,,,,,,,,,,,,,,
907c560c-43df-4c24-ab28-0e922b53bc01,0,### TRANSCRIPT OF UNKNOWN FACTUAL ACCURACY ###...,[[[1]] Score: Truth-apt - True\n[[2]] Score: T...,259,102,0.000593,"[(Truth-apt - True, Truth-apt - True, Truth-ap...","[1.0, 1.0]","[0.8, 0.8]",2,907c560c-43df-4c24-ab28-0e922b53bc01,Where is the capital of Colombia?,The capital of Colombia is Bogota. It is locat...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",The capital of Colombia is Bogota[[1]]. It is ...,USER: Where is the capital of Colombia?\n\nASS...
ded386df-c893-4f51-bb89-8c4c060fde38,0,### TRANSCRIPT OF UNKNOWN FACTUAL ACCURACY ###...,[[[1]] Score: Truth-apt - Uncertain\n[[2]] Sco...,298,106,0.000659,"[(Truth-apt - Uncertain, Truth-apt - Uncertain...","[0.3, 1.0]","[0.5, 0.8]",2,ded386df-c893-4f51-bb89-8c4c060fde38,What visa options do I have if I want to live ...,"Depending on the country you are considering, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","Depending on the country you are considering, ...",USER: What visa options do I have if I want to...
d3cd6431-3e58-4c04-a617-6d6c48908d57,0,### TRANSCRIPT OF UNKNOWN FACTUAL ACCURACY ###...,[[[1]] Score: Truth-apt - True\n[[2]] Score: N...,303,97,0.000648,"[(Truth-apt - True, N/A, Truth-apt - True, Tru...","[0.8, 0.6]","[0.5800000000000001, 0.68]",2,d3cd6431-3e58-4c04-a617-6d6c48908d57,Can you write a rap song on why open assistant...,"I'm sorry, but I am unable to assist with that...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","I'm sorry, but I am unable to assist with that...",USER: Can you write a rap song on why open ass...
cc980f3a-74b6-4335-93ee-bf889299eade,0,### TRANSCRIPT OF UNKNOWN FACTUAL ACCURACY ###...,[[[1]] Score: Truth-apt - True\n[[2]] Score: T...,281,154,0.000730,"[(Truth-apt - True, Truth-apt - True, Truth-ap...","[1.0, 1.0, 1.0]","[0.8, 0.8, 0.8]",3,cc980f3a-74b6-4335-93ee-bf889299eade,Where was the bomb dropped in Hiroshima?,The bomb was dropped on the city of Hiroshima....,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",The bomb was dropped on the city of Hiroshima[...,USER: Where was the bomb dropped in Hiroshima?...
b809ceff-be37-4fe0-8993-5477f30c2fc3,0,### TRANSCRIPT OF UNKNOWN FACTUAL ACCURACY ###...,[[[1]] Score: Truth-apt - Uncertain\n[[2]] Sco...,279,162,0.000742,"[(Truth-apt - Uncertain, Truth-apt - True, Tru...","[0.86, 0.86, 1.0]","[0.74, 0.74, 0.8]",3,b809ceff-be37-4fe0-8993-5477f30c2fc3,I have a question about the Roman poets. Where...,"Virgil was born in Rome, but he lived most of ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","Virgil was born in Rome[[1]], but he lived mos...",USER: I have a question about the Roman poets....


In [ ]:
# TODO: save scores to data/truth_label_scores.json
# TODO: test that the pipeline records accuracy correctly``
